### Airdata Dataset

In [42]:
import pandas as pd 
import numpy as np 

In [43]:
data = pd.read_excel("updated_dataset/filtered_dataset.xlsx")
data.head()

,date,time,stationId,code,value,season
0,2023-04-01,02:00:00,k12,CO,200.406064,heating
1,2023-04-01,02:00:00,k12,NO,0.000000,heating
2,2023-04-01,02:00:00,k12,PM10,248.321723,heating
3,2023-04-01,02:00:00,k12,SO2,31.540874,heating
4,2023-04-01,02:00:00,k7,CO,469.650056,heating


In [44]:
data['stationId'].unique()

array(['k12', 'k7', 'k8', 'k9'], dtype=object)

In [45]:
data['code'].unique()

array(['CO', 'NO', 'PM10', 'SO2', 'PM2.5'], dtype=object)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56944 entries, 0 to 56943
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       56944 non-null  datetime64[ns]
 1   time       56944 non-null  object        
 2   stationId  56944 non-null  object        
 3   code       56944 non-null  object        
 4   value      56944 non-null  float64       
 5   season     56944 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 2.6+ MB


### Station Dataset

In [48]:
station = pd.read_excel("updated_dataset/station_address.xlsx")

In [49]:
station.head()

,address,type,fuel,position,lat,lng,power
0,"Zh. Zhabaeva, 23/1",“Astana-Energy” JSC,Municipal coal,District boilers,51.144190,71.568258,"5,65 Gcal/h"
1,"Shalkozhe, 1/3",“Astana-Energy” JSC,Municipal coal,District boilers,51.122690,71.520934,"5,11 Gcal/h"
2,"Armandastar, 2/3",“Astana-Energy” JSC,Municipal coal,District boilers,51.129190,71.589700,"2,49 Gcal/h"
3,Kokil 1,“Astana-Energy” JSC,Municipal coal,District boilers,51.114237,71.645135,"5,11 Gcal/h"
4,"Arnasay, 151",“Astana-Energy” JSC,Municipal coal,District boilers,51.041822,71.431786,"5,15 Gcal/h"


In [50]:
station['address'].unique()

array(['Zh. Zhabaeva, 23/1', 'Shalkozhe, 1/3', 'Armandastar, 2/3',
       'Kokil 1', 'Arnasay, 151', 'E-522, 15T',
       'Myrzhakyp Dulatov, 182/1', 'Koktal-1, Sulukol 14',
       'Koktal-1 intersection of Shugyla Street and No. 12-96',
       'Е-495,\xa08', 'Е-321, 22', 'Е-319, 2', 'Е-319, 2А', 'Е-319, 2B',
       'Е-652, 2', 'Е-652, 4', 'Е-652, 6', 'Е-652, 8', 'Е-652, 10',
       'Е-652, 12', 'Е-652, 14', 'Е-652, 16',
       'Kurgalzhinskoye highway, 28', 'Orynbor, 22',
       'Kayym Mukhamedkhanov, 41', 'Е-496, 10', 'Е-496, 10/1',
       'Е-496, 10/2', 'Ilyas Omarov, 23/1',
       'Kurgalzhinskoye highway, 110', 'Е-535',
       'Residential Complex “Astana”, “Urker” (11 houses)',
       'Residential Complex “Kamshat” E-535',
       'Kurgalzhinskoye highway, 30',
       'Residential Complex Budapest, Kabanbay Batyr, 59B',
       'Kabanbai Batyr, 75A', 'Sarytogai, 2a', 'Sarytogai, 11',
       'Belasar, 2A', 'Maymeken, 2', 'Maymeken, 4', 'Maymeken, 6',
       'Maymeken, 6/1', 'Dulata 

In [51]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   address   95 non-null     object 
 1   type      91 non-null     object 
 2   fuel      91 non-null     object 
 3   position  95 non-null     object 
 4   lat       95 non-null     float64
 5   lng       95 non-null     float64
 6   power     56 non-null     object 
dtypes: float64(2), object(5)
memory usage: 5.3+ KB


# Merging Weather dataset

In [53]:
import pandas as pd
import os
from datetime import datetime, timedelta

data["date"] = data["date"].astype(str)
data["time"] = data["time"].astype(str)

data["datetime"] = pd.to_datetime(data["date"] + " " + data["time"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

weather_folder = "weatherapi/new"

merged_data = []

for station_id in data["stationId"].unique():
    station_file = os.path.join(weather_folder, f"{station_id}.csv")
    
    if os.path.exists(station_file):
        weather_df = pd.read_csv(station_file)
        weather_df["datetime"] = pd.to_datetime(weather_df["date"], errors='coerce', utc=True) + timedelta(hours=6)
        weather_df["datetime"] = weather_df["datetime"].dt.tz_localize(None)
        weather_df = weather_df[weather_df["datetime"] >= "2023-04-01 00:00:00"]

        station_data = data[data["stationId"] == station_id]
        merged = station_data.merge(weather_df, on="datetime", how="left")
        merged_data.append(merged)
# Concatenate all merged data
final_df = pd.concat(merged_data, ignore_index=True)

final_df

,date_x,time,stationId,code,value,season,datetime,date_y,temperature_2m,relative_humidity_2m,precipitation,pressure_msl,wind_speed_10m,wind_direction_10m,cloud_cover
0,2023-04-01,02:00:00,k12,CO,200.406064,heating,2023-04-01 02:00:00,2023-03-31 20:00:00+00:00,-4.3825,57.67535,0.0,1039.4,3.801315,116.56499,0.0
1,2023-04-01,02:00:00,k12,NO,0.000000,heating,2023-04-01 02:00:00,2023-03-31 20:00:00+00:00,-4.3825,57.67535,0.0,1039.4,3.801315,116.56499,0.0
2,2023-04-01,02:00:00,k12,PM10,248.321723,heating,2023-04-01 02:00:00,2023-03-31 20:00:00+00:00,-4.3825,57.67535,0.0,1039.4,3.801315,116.56499,0.0
3,2023-04-01,02:00:00,k12,SO2,31.540874,heating,2023-04-01 02:00:00,2023-03-31 20:00:00+00:00,-4.3825,57.67535,0.0,1039.4,3.801315,116.56499,0.0
4,2023-04-01,05:00:00,k12,CO,271.849180,heating,2023-04-01 05:00:00,2023-03-31 23:00:00+00:00,-5.1325,60.54702,0.0,1039.1,3.275668,121.26367,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56939,2024-03-31,23:00:00,k9,CO,354.045000,heating,2024-03-31 23:00:00,2024-03-31 17:00:00+00:00,-2.2370,86.48255,0.0,1031.6,3.189044,228.81418,0.0
56940,2024-03-31,23:00:00,k9,NO,3.448333,heating,2024-03-31 23:00:00,2024-03-31 17:00:00+00:00,-2.2370,86.48255,0.0,1031.6,3.189044,228.81418,0.0
56941,2024-03-31,23:00:00,k9,PM10,15.803667,heating,2024-03-31 23:00:00,2024-03-31 17:00:00+00:00,-2.2370,86.48255,0.0,1031.6,3.189044,228.81418,0.0
56942,2024-03-31,23:00:00,k9,PM2.5,9.535333,heating,2024-03-31 23:00:00,2024-03-31 17:00:00+00:00,-2.2370,86.48255,0.0,1031.6,3.189044,228.81418,0.0


In [54]:
final_df.drop(columns=["date_x", "date_y", "time"], inplace=True)
final_df["datetime"] = pd.to_datetime(final_df["datetime"], errors='coerce')

Checking for duplicates of the same datetime x stationId x code combination

In [56]:
import pandas as pd

duplicates = final_df[final_df.duplicated(subset=['stationId', 'datetime', 'code'], keep=False)]
print(duplicates)

      stationId   code        value   season            datetime  \
12179       k12     CO   270.698949  heating 2024-03-05 02:00:00   
12180       k12     NO     0.000000  heating 2024-03-05 02:00:00   
12181       k12   PM10    31.094674  heating 2024-03-05 02:00:00   
12182       k12  PM2.5    30.833439  heating 2024-03-05 02:00:00   
12183       k12     CO   609.108477  heating 2024-03-05 02:00:00   
12184       k12     NO     0.000000  heating 2024-03-05 02:00:00   
12185       k12   PM10    57.556718  heating 2024-03-05 02:00:00   
12186       k12  PM2.5    55.667855  heating 2024-03-05 02:00:00   
52900        k9     CO   399.459310  heating 2023-12-22 23:00:00   
52901        k9     NO     0.000000  heating 2023-12-22 23:00:00   
52902        k9   PM10    20.090019  heating 2023-12-22 23:00:00   
52903        k9  PM2.5    20.032539  heating 2023-12-22 23:00:00   
52939        k9     CO  1187.163889  heating 2023-12-22 23:00:00   
52940        k9     NO     0.000000  heating 202

Since there appear to be different values for one pollutant x station x datetime, we will take average of 2 nearest entries nd calculate average

In [58]:
import pandas as pd

df = final_df

df['datetime'] = pd.to_datetime(df['datetime'])

problematic_rows = [
    ('k9', '2023-12-22 23:00:00'),
    ('k9', '2024-03-02 08:00:00'),
    ('k12', '2024-03-05 02:00:00')
]

for station, dt in problematic_rows:
    dt = pd.Timestamp(dt)  

    for code in df[df['stationId'] == station]['code'].unique():  
        past_value = df[(df['stationId'] == station) & (df['code'] == code) & 
                        (df['datetime'] < dt)].sort_values('datetime', ascending=False).head(1)['value']

        future_value = df[(df['stationId'] == station) & (df['code'] == code) & 
                          (df['datetime'] > dt)].sort_values('datetime', ascending=True).head(1)['value']

        past_avg = past_value.mean() if not past_value.empty else None
        future_avg = future_value.mean() if not future_value.empty else None

        avg_value = None
        if past_avg is not None and future_avg is not None:
            avg_value = (past_avg + future_avg) / 2
        elif past_avg is not None:
            avg_value = past_avg
        elif future_avg is not None:
            avg_value = future_avg

        df.loc[(df['stationId'] == station) & (df['datetime'] == dt) & (df['code'] == code), 'value'] = avg_value


In [59]:
df = df.drop_duplicates(subset=['stationId', 'datetime', 'code'], keep='first')

In [60]:
import pandas as pd

pivot_values = df.pivot(index=['stationId', 'datetime'], columns='code', values='value').reset_index()

weather_columns = ["temperature_2m", "relative_humidity_2m", "season",	"precipitation",	"pressure_msl", 	"wind_speed_10m", "wind_direction_10m", "cloud_cover"]  # Add other weather-related columns here
weather_data = df.drop_duplicates(subset=['stationId', 'datetime'])[['stationId', 'datetime'] + weather_columns]

final_df = pivot_values.merge(weather_data, on=['stationId', 'datetime'])

final_df

,stationId,datetime,CO,NO,PM10,PM2.5,SO2,temperature_2m,relative_humidity_2m,season,precipitation,pressure_msl,wind_speed_10m,wind_direction_10m,cloud_cover
0,k12,2023-04-01 02:00:00,200.406064,0.000000,248.321723,NaN,31.540874,-4.3825,57.675350,heating,0.0,1039.4,3.801315,116.564990,0.0
1,k12,2023-04-01 05:00:00,271.849180,0.000000,NaN,NaN,48.657248,-5.1325,60.547020,heating,0.0,1039.1,3.275668,121.263670,0.0
2,k12,2023-04-01 08:00:00,353.344343,0.000000,NaN,NaN,39.866436,-3.8825,59.912790,heating,0.0,1040.1,3.324154,111.161330,0.0
3,k12,2023-04-01 11:00:00,286.364430,0.000000,NaN,NaN,31.369603,2.3175,37.652370,heating,0.0,1039.2,4.701064,113.838745,0.0
4,k12,2023-04-01 14:00:00,227.553338,0.000000,247.957699,NaN,48.619929,6.3175,26.806433,heating,0.0,1037.3,5.630276,109.722374,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11704,k9,2024-03-31 11:00:00,260.350389,14.446389,14.001278,8.044611,23.695167,0.4630,65.595850,heating,0.0,1029.0,4.517743,242.300610,12.0
11705,k9,2024-03-31 14:00:00,254.334500,4.083556,7.373944,4.519889,23.419389,4.7130,53.500020,heating,0.0,1029.1,4.085339,248.458950,16.0
11706,k9,2024-03-31 17:00:00,302.164444,5.538111,11.612111,8.955000,24.855556,6.5630,45.845260,heating,0.0,1029.1,4.742362,245.056120,4.0
11707,k9,2024-03-31 20:00:00,347.951167,2.768111,13.508556,4.433944,22.908889,-0.4370,83.206430,heating,0.0,1031.5,4.101220,224.999900,2.0


In [61]:
import pandas as pd

df = final_df

pollutant_cols = ['CO', 'NO', 'PM10', 'PM2.5', 'SO2']

# Count non-null values per station for each pollutant
pollutant_counts = df.groupby('stationId')[pollutant_cols].apply(lambda x: x.notna().sum()).reset_index()

# Count total number of entries per stationId
total_entries = df.groupby('stationId').size().reset_index(name='total_entries')

# Merge the pollutant counts with total entry count
pollutant_counts = pollutant_counts.merge(total_entries, on='stationId')

print(pollutant_counts)


  stationId    CO    NO  PM10  PM2.5   SO2  total_entries
0       k12  2927  2927  2584   2383  2221           2927
1        k7  2928  2928  2928   2928  2928           2928
2        k8  2928  2928  2927   2927  2928           2928
3        k9  2926  2926  2926   2926  2907           2926


In [62]:

# Convert 'season' to a binary column 'is_heating'
final_df['is_heating'] = (final_df['season'] == "heating").astype(int)

# Drop the old 'season' column
final_df = final_df.drop(columns=['season'])

print(final_df.head())  # Check the updated DataFrame


  stationId            datetime          CO   NO        PM10  PM2.5  \
0       k12 2023-04-01 02:00:00  200.406064  0.0  248.321723    NaN   
1       k12 2023-04-01 05:00:00  271.849180  0.0         NaN    NaN   
2       k12 2023-04-01 08:00:00  353.344343  0.0         NaN    NaN   
3       k12 2023-04-01 11:00:00  286.364430  0.0         NaN    NaN   
4       k12 2023-04-01 14:00:00  227.553338  0.0  247.957699    NaN   

         SO2  temperature_2m  relative_humidity_2m  precipitation  \
0  31.540874         -4.3825             57.675350            0.0   
1  48.657248         -5.1325             60.547020            0.0   
2  39.866436         -3.8825             59.912790            0.0   
3  31.369603          2.3175             37.652370            0.0   
4  48.619929          6.3175             26.806433            0.0   

   pressure_msl  wind_speed_10m  wind_direction_10m  cloud_cover  is_heating  
0        1039.4        3.801315          116.564990          0.0           1  


In [71]:
final_df["datetime"] = pd.to_datetime(final_df["datetime"])
final_df["hour"] = final_df["datetime"].dt.hour
final_df["day_of_week"] = final_df["datetime"].dt.weekday
final_df['month'] = final_df["datetime"].dt.month
final_df["wind_x"] = final_df["wind_speed_10m"] * np.cos(np.radians(final_df["wind_direction_10m"]))
final_df["wind_y"] = final_df["wind_speed_10m"] * np.sin(np.radians(final_df["wind_direction_10m"]))

In [75]:
def get_season(final_df):
    month = final_df.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'
        
final_df['season'] = final_df['datetime'].apply(get_season)
final_df = pd.get_dummies(final_df, columns=['season'], dtype=int)

final_df.head()


,stationId,datetime,CO,NO,PM10,PM2.5,SO2,temperature_2m,relative_humidity_2m,precipitation,...,is_heating,hour,day_of_week,month,wind_x,wind_y,season_Fall,season_Spring,season_Summer,season_Winter
0,k12,2023-04-01 02:00:00,200.406064,0.0,248.321723,NaN,31.540874,-4.3825,57.675350,0.0,...,1,2,5,4,-1.699996,3.400002,0,1,0,0
1,k12,2023-04-01 05:00:00,271.849180,0.0,NaN,NaN,48.657248,-5.1325,60.547020,0.0,...,1,5,5,4,-1.699997,2.800002,0,1,0,0
2,k12,2023-04-01 08:00:00,353.344343,0.0,NaN,NaN,39.866436,-3.8825,59.912790,0.0,...,1,8,5,4,-1.200004,3.099999,0,1,0,0
3,k12,2023-04-01 11:00:00,286.364430,0.0,NaN,NaN,31.369603,2.3175,37.652370,0.0,...,1,11,5,4,-1.900000,4.300000,0,1,0,0
4,k12,2023-04-01 14:00:00,227.553338,0.0,247.957699,NaN,48.619929,6.3175,26.806433,0.0,...,1,14,5,4,-1.900009,5.299997,0,1,0,0


In [77]:
final_df["stationId"].unique()

array(['k12', 'k7', 'k8', 'k9'], dtype=object)

In [83]:
final_df = pd.get_dummies(final_df, columns=['stationId'], prefix='station')

final_df.head()

,datetime,CO,NO,PM10,PM2.5,SO2,temperature_2m,relative_humidity_2m,precipitation,pressure_msl,...,wind_x,wind_y,season_Fall,season_Spring,season_Summer,season_Winter,station_k12,station_k7,station_k8,station_k9
0,2023-04-01 02:00:00,200.406064,0.0,248.321723,NaN,31.540874,-4.3825,57.675350,0.0,1039.4,...,-1.699996,3.400002,0,1,0,0,True,False,False,False
1,2023-04-01 05:00:00,271.849180,0.0,NaN,NaN,48.657248,-5.1325,60.547020,0.0,1039.1,...,-1.699997,2.800002,0,1,0,0,True,False,False,False
2,2023-04-01 08:00:00,353.344343,0.0,NaN,NaN,39.866436,-3.8825,59.912790,0.0,1040.1,...,-1.200004,3.099999,0,1,0,0,True,False,False,False
3,2023-04-01 11:00:00,286.364430,0.0,NaN,NaN,31.369603,2.3175,37.652370,0.0,1039.2,...,-1.900000,4.300000,0,1,0,0,True,False,False,False
4,2023-04-01 14:00:00,227.553338,0.0,247.957699,NaN,48.619929,6.3175,26.806433,0.0,1037.3,...,-1.900009,5.299997,0,1,0,0,True,False,False,False


In [85]:
final_df.to_csv("airdata_updated.csv")